In [5]:
import os
import fitz  # PDF 텍스트 추출
import pandas as pd
from dotenv import load_dotenv
import chromadb
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnableMap
from langchain_core.output_parsers import StrOutputParser

In [2]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
CHROMA_HOST = os.getenv("CHROMA_HOST")
CHROMA_PORT = os.getenv("CHROMA_PORT")

In [ ]:
df = pd.read_csv('./data/job_opening.csv')
rec_df = pd.read_csv('../data_backup/rec_data.csv')

In [ ]:
rec_df.drop(columns='Unnamed: 0', inplace=True)

In [ ]:
# 위 코드가 분당 토큰 100만을 넘겼기에 사용
from time import sleep

batch_size = 500
batch_documents = [documents[i:i+batch_size] for i in range(0, len(documents), batch_size)]

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

vectordb = None

for i, batch in enumerate(batch_documents):
    print(f"▶ Processing batch {i+1}/{len(batch_documents)}...")
    if i == 0:
        vectordb = Chroma.from_documents(
            documents=batch,
            embedding=embedding_model,
            persist_directory="./chroma_data",
            collection_name="chroma_test"
        )
    else:
        vectordb.add_documents(batch)

    vectordb.persist()
    sleep(65)


In [3]:
# 이력서 불러오기
path = "./data/빅데이터AI_이력서.pdf"
doc = fitz.open(path)
resume_text=''
for page in doc:
    resume_text += page.get_text()
print(resume_text)

📄 이력서 (Resume)
[기본 정보]
이름: 홍길동
연락처: 010-1234-5678
이메일: honggildong.ai@gmail.com
주소: 서울특별시 강남구 테헤란로 123
[학력]
고려대학교 컴퓨터학과 졸업 (2018.03 ~ 2024.02)
GPA: 3.85 / 4.5
관련 과목: 머신러닝, 데이터마이닝, 통계학, 빅데이터처리, 딥러닝 이론과 실습
[기술 스택]
Programming: Python, SQL, R
Frameworks/Libraries: Scikit-learn, TensorFlow, PyTorch, Pandas, NumPy
Tools: Jupyter, Git, Docker, Tableau
DBMS: MySQL, MongoDB, Hadoop(HDFS), Spark
Cloud: Google Colab, AWS EC2 & S3 (기초 수준)
[프로젝트 경험]
1. 신문 기사 기반 감성 분석 모델 개발 (2023.03 ~ 2023.06)
자연어처리(NLP) 기반 감성 분류 모델 개발
KoNLPy와 Scikit-learn을 이용한 전처리 및 모델 학습
정확도 86% 달성
2. 머신러닝 기반 개인 맞춤형 영화 추천 시스템 (2023.09 ~ 2023.12)
📄 이력서 (Resume)
1
Content-based Filtering 및 Collaborative Filtering 기법 적용
Streamlit으로 웹 인터페이스 구현
kaggle 데이터셋 기반, Precision@10: 0.73
[자격증]
ADsP (데이터분석 준전문가) – 2023.08
SQLD (SQL 개발자) – 2024.01
📄 이력서 (Resume)
2



In [ ]:
# resume_text :이력서 텍스트 입력 쿼리가 될 것

# df : 채용공고 원본 데이터

# document 구조
# page_content  
# 기업명, 공고명, [경력]
# 직무, 지역 
# 임베딩 되야할건 jd_text

In [ ]:
# documents 생성

documents = [
    Document(
        page_content=(
            f"공고내용: {row['chunk']}, "
            f"회사명: {row['company_nm']}, "
            f"직무: {row['recruit_kewdcdnm']}, "
            f"지역: {row['company_place']}, "
            f"경력: {row['career']}, "
            f"학력: {row['education']}"
        ),
        metadata={"rec_idx": row['rec_idx'], "company_nm":}
    )
    for _, row in rec_df.iterrows()
]

In [7]:
# EC2 chroma data base에 적재
import chromadb
from chromadb.config import Settings
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

# 1. 임베딩 모델
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 2. Chroma EC2 클라이언트 연결
chroma_client = chromadb.HttpClient(
    host="43.202.186.183",
    port=8000,
    settings=Settings(allow_reset=True, anonymized_telemetry=False)
)

/tmp/ipykernel_295109/1344237561.py:8: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [ ]:
# 3. 문서 임베딩 + 적재 (cosine distance)
chroma_db = Chroma.from_documents(
    documents=documents,  # rec_df에서 생성한 Document 리스트
    embedding=embeddings,
    client=chroma_client,
    collection_name="job_position",
    collection_metadata={"hnsw:space": "cosine"}  # cosine similarity 사용
)

In [ ]:
# 적재 부분에서 너무 커서 임베딩 한계 토큰을 넘음 time 옵션 넣기

import time
import logging
from tenacity import retry, wait_random_exponential, stop_after_attempt
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

# 재시도 로직, 로깅 설정
logging.basicConfig(level=logging.DEBUG)

@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def embed_with_retry(docs):
    try:
        logging.debug(f"Processing batch with {len(docs)} documents.")
        return Chroma.from_documents(
            documents=docs,
            embedding=embeddings,
            client=chroma_client,
            collection_name="job_position",
            collection_metadata={"hnsw:space": "cosine"},
        )
    except Exception as e:
        logging.error(f"Error in embedding: {e}")
        raise
batch_size = 50  # 작은 배치

In [ ]:
for i in range(0, len(documents), batch_size):
    batch = documents[i:i+batch_size]
    try:
        embed_with_retry(batch)
        print(f"{i} ~ {i+batch_size} 적재 완료")
        time.sleep(2)  # 호출 간 간단한 지연 추가
    except Exception as e:
        print(f"에러 발생 - {i} ~ {i+batch_size}번째 문서 배치: {e}")

In [8]:
chroma_client = chromadb.HttpClient(
    host="43.202.186.183",
    port=8000,
    settings=Settings(allow_reset=True, anonymized_telemetry=False)
)
collection = chroma_client.get_collection(name="job_position")

In [ ]:
results = collection.get()
print("총 문서 수:", len(results['documents']))
print("예시 문서:", results['documents'][0])

In [10]:
# chroma 유사도 검색 테스트 
# Chroma 래퍼를 통해 기존 컬렉션 불러오기
chroma_db = Chroma(
    collection_name="job_position",
    embedding_function=embeddings,
    client=chroma_client
)

# similarity_search 수행
query = "python backend developer"
results = chroma_db.similarity_search(query, k=5)

for i, res in enumerate(results):
    print(f"\n[{i+1}] {res.page_content}")

/tmp/ipykernel_295109/3214207519.py:3: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chroma_db = Chroma(



[1] 공고내용: [보안AI사업본부] Python 백엔드 개발자 모집 채용공고 상세
[보안AI사업본부] Python 백엔드 개발자 모집2008년 02월 15일에 설립된 응용 소프트웨어 개발 및 공급업업종의 의료 인공지능플랫폼,인공지능 임상의사결정 시스템 개발사업을 하는 코스닥,중소기업,외부감사법인,주식회사,병역특례 인증업체기업 입니다.모집부문 및 상세내용구   분상세내용공통 자격요건ㆍ학력 : 대졸 이상 (4년) / 컴퓨터 공학 또는 관련 전공자ㆍ경력 : 3년 이상 Python 백엔드 개발자 1명주요업무담당업무ㆍ담당업무ㆍPython 제품 안정화 개발 및 배포ㆍ영상 분석 AI와 제품 간 인터페이스 개발 담당자격요건ㆍPython 개발자 (프로젝트 포트폴리오 제출 필수)ㆍFastAPI와 같은 Python 웹 프레임워크 활용 경험우대사항ㆍPython을 통한 영상 AI 솔루션 개발 유 경험자ㆍPython 솔루션 개발 및 배포 유 경험자ㆍWindows OS에서 PyInstaller를 통한 배포 환경 경험 및 배포 관련 개선 경험자ㆍAI 모델 Docker 배포 경험자ㆍPython 외에 다양한 개발언어 활용 가능자보유자격 (필수는 아님 보유 시 우대)ㆍ정보처리기사, 운전면허 등 필수 제출 서류ㆍ면접 전 포트폴리오 제출 필수, 면접 시 포트폴리오 발표 진행 예정
ㆍ기타 필수 사항
우대사항근무조건ㆍ근무형태:정규직(수습기간)-3개월ㆍ근무일시:ㆍ근무지역:(08377) 서울 구로구 디지털로33길 48 대륭포스트타워7차 19층(구로동) - 서울 7호선 남구로 에서 800m 이내전형절차 서류전형 1차면접 2차면접 최종합격접수기간 및 방법ㆍ접수기간:2025년 3월 18일 (화) 21시~ 채용시ㆍ접수방법:사람인 입사지원ㆍ이력서양식:사람인 온라인 이력서ㆍ제출서류:포트폴리오 제출 필수 / 면접 시 포트폴리오 발표 진행 예정유의사항ㆍ학력, 성별, 연령을 보지않는 블라인드 채용입니다. ㆍ입사지원 서류에 허위사실이 발견될 경우, 채용확정 이후라도 채용이 취소될 수 있습니다.ㆍ모집분야별로 마감일이 상

# 엘라스틱 서치 적재 시작

In [9]:
from elasticsearch import Elasticsearch, helpers
from langchain.vectorstores import ElasticsearchStore
import uuid

# Elasticsearch 연결 설정
es_client = Elasticsearch ("http://43.202.186.183:9200", basic_auth=("elastic", "ElastiC7276" ))

In [11]:
elasticsearch_store = ElasticsearchStore(
    es_connection=es_client,               # ← 여기서 네 es_client 사용
    index_name="job_position",
    embedding=embeddings,
)

/tmp/ipykernel_295109/1471931124.py:1: LangChainPendingDeprecationWarning: The class `ElasticsearchStore` will be deprecated in a future version. Use :class:`~Use class in langchain-elasticsearch package` instead.
  elasticsearch_store = ElasticsearchStore(


In [12]:
es_client.info()

ObjectApiResponse({'name': 'e350c3ef7c8e', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'LvpeIMa2RE21sK2gD6tyBA', 'version': {'number': '8.17.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '747663ddda3421467150de0e4301e8d4bc636b0c', 'build_date': '2025-02-05T22:10:57.067596412Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def embed_and_store(batch_docs):
    ElasticsearchStore.from_documents(
        documents=batch_docs,
        embedding=embeddings,
        index_name="job_position",
        es_connection=es_client,
    )

batch_size = 50  # 적절한 배치 사이즈 (20~50 추천)

for i in range(0, len(documents), batch_size):
    batch = documents[i:i+batch_size]
    try:
        embed_and_store(batch)
        print(f"{i} ~ {i+batch_size} 적재 완료")
        time.sleep(2)  # 너무 몰아서 호출하지 않도록
    except Exception as e:
        print(f"❌ {i} ~ {i+batch_size} 배치 적재 실패: {e}")

# 여기서부터 시작

In [13]:
chroma_retriever = chroma_db.as_retriever(search_kwargs={"k": 5})
es_retriever = elasticsearch_store.as_retriever(search_kwargs={"k": 5})

In [21]:
from langchain.retrievers import EnsembleRetriever

hybrid_retriever = EnsembleRetriever(
    retrievers=[chroma_retriever, es_retriever],
    weights=[0.9, 0.1],
)

In [15]:
query = resume_text
response = hybrid_retriever.invoke(query)

In [ ]:
print("[hybrid_retriever]")
for doc in response:
    print(f"Content: {doc.page_content}")
    print()

In [23]:
rag_prompt = PromptTemplate.from_template(
    """
    당신은 인재매칭 AI 어시스턴트입니다. 사용자 이력서에 기반하여 채용공고를 5개 추천해주세요.
    
    - 반드시 이력서에 기반할 것.
    - 출력 시 채용공고의 양식을 사용할 것.
    - 출력 시 지역은 상세하게 출력 할 것.
    - 채용공고 추천 이유를 한줄로 설명할 것.         
    - 같은 공고 번호가 중복될 경우 단 1개만 추천할 것.                  
    - 채용공고 전체 내용을 기반하여 분석할 것. 제목만 보고 판단하지 말 것.
    - page_content의 전체 텍스트를 기준으로 판단할 것.

    #이력서: 
    {question} 
    #채용공고: 
    {context} 

    #출력형태
    - 기업명, 공고명, [경력]
    - 직무, 지역 
    """
)

llm = ChatOpenAI(model_name="gpt-4o", temperature=1)

hybrid_chain = (
    {"context": hybrid_retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

    #- 추천된 [채용공고]와 [이력서]를 참고하여 각 공고 별로 예상 면접 질문 5개를 만들 것.
    #    - 면접 질문 유형은 아래와 같이 출력할 것.
    #    - 1개 회사와 관련된 질문
    #    - 2개 인적성 면접
    #    - 2개 기술/역량 면접
    #- 면접 질문은 웹 검색을 허용

In [24]:
response = hybrid_chain.invoke(resume_text)
print(response)

- (주)유니와이즈솔루션즈, 연구소_AI 교육 정부 과제 연구원_경력
  - 직무: ['데이터분석가', '데이터엔지니어', '데이터마이닝', '데이터시각화', '모델링'], 지역: 서울 강남구
  - 추천 이유: AI 및 데이터 분석 관련 실무 경험과 Python 및 AI 프레임워크 지식이 요구되는 직무로, 사용자의 경력과 기술 스택과 잘 맞습니다.

- (주)씨엘모빌리티, [강남] AI 연구소 개발 연구원 채용 (상시)
  - 직무: ['딥러닝', '머신러닝', '연구원'], 지역: 서울 강남구 외
  - 추천 이유: 딥러닝과 머신러닝 분야에 대한 깊은 이해와 경험이 요구되며, 사용자의 프로젝트 경험과 일치합니다.

- (주)크랜베리, [경력] 파이썬 개발자 채용
  - 직무: ['데이터분석가', '데이터엔지니어', '백엔드/서버개발', '앱개발', '웹개발'], 지역: 서울 금천구 외
  - 추천 이유: Python과 AI 모델 운영 경험이 요구되며, 사용자의 기술과 경력을 효과적으로 활용할 수 있습니다.

- (주)코난테크놀로지, 강화학습 개발자 채용
  - 직무: ['딥러닝', '머신러닝', '모델링'], 지역: 서울 서초구
  - 추천 이유: 파이썬과 강화학습 모델 개발 경험이 요구되며, 사용자의 학력과 프로젝트 경험이 이를 뒷받침합니다.

- 한국과학기술연구원, 2025년 3월 연수직(연구인턴) 공개채용
  - 직무: ['머신러닝', '모델링', '자율주행', '컴퓨터비전', 'AI(인공지능)'], 지역: 서울 성북구 외
  - 추천 이유: 머신러닝과 AI 프로젝트 경험이 요구되며, 이는 사용자의 학력 및 프로젝트 경험과 잘 부합합니다.


In [20]:
for chunk in hybrid_chain.stream(resume_text):
    print(chunk, end="", flush=True)

### 채용공고 추천 목록

1. **기업명**: (주)크랜베리  
   **공고명**: [경력] 파이썬 개발자 채용  
   **직무**: 데이터분석가, 데이터엔지니어, 백엔드/서버개발, 앱개발, 웹개발  
   **지역**: 서울 금천구 가산디지털1로 225 에이스 가산 포휴 316~318호  

   **추천 이유**: 홍길동님의 파이썬 및 AI 모델 개발 경험이 회사의 요구사항에 부합합니다.  
   **예상 면접 질문**:
   - **회사 관련**: (주)크랜베리의 AI 운영 시스템 개발에 어떻게 기여할 수 있을까요?
   - **인적성 면접**:
     1. 자신의 문제 해결 능력을 보여준 경험에 대해 이야기해 주세요.
     2. 팀 내의 갈등을 해결한 경험이 있습니까? 어떻게 해결했나요?
   - **기술/역량 면접**:
     1. 파이썬을 사용하여 개발한 프로젝트가 있다면 설명해 주세요.
     2. 대규모 데이터 핸들링 경험에 대해 설명해 주세요.

2. **기업명**: (주)유니와이즈솔루션즈  
   **공고명**: 연구소_AI 교육 정부 과제 연구원_경력  
   **직무**: 데이터분석가, 데이터엔지니어, 데이터마이닝, 데이터시각화, 모델링  
   **지역**: 서울 강남구 광평로 295, 2층  

   **추천 이유**: 데이터 분석 및 AI 프레임워크에 대한 지식이 요구되는 직무로서, 홍길동님의 기술 스택과 일치합니다.  
   **예상 면접 질문**:
   - **회사 관련**: (주)유니와이즈솔루션즈에서 AI 교육 연구를 진행할 때 중점을 두고 싶은 부분은 무엇인가요?
   - **인적성 면접**:
     1. 데이터 분석에서 가장 중요한 요소는 무엇이라고 생각하나요?
     2. 본인의 장단점에 대해 설명해 주세요.
   - **기술/역량 면접**:
     1. TensorFlow나 PyTorch를 활용한 프로젝트 경험이 있나요? 설명해 주세요.
     2. AI 모델의 최적화 작업을 수행한 경험이 있다면 

In [ ]:
print(resume_text)